In [1]:
import pandas_datareader.data as web #to collect data
import datetime as dt #to specify start and end dates

# import yfinance as yf

import eventstudy as es
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


import pandas as pd

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.regression.rolling import RollingOLS

from patsy import dmatrices
from tqdm.notebook import tqdm
tqdm.pandas()

pd.set_option('display.max_columns', 500)

In [2]:
import_folder_path = "car_output1"
output_folder_path = "car_output2"
supporting_folder_path = "supporting_datafiles"

## 120 CAR Data Reading

In [3]:
ols120 = pd.read_pickle(rf"{import_folder_path}\ols120_2.pkl")

## 120 CAR Calc

In [4]:
ols120CAR = ols120.sort_values(by = ["CompanyName", "AsOnDate"])
ols120CAR

,CompanyName,ProwessCode,Symbol,AsOnDate,ACP,pct,RF,RMRF,MF,SMB,HML,OLS120_intercept,OLS120_RMRF,OLS120_r_squared,OLS120_adjusted_r_squared,OLS120_f_p_value
0,20 Microns Ltd.,11.0,20MICRONS,2008-10-06,16.82,NaN,0.069713,-6.381449,-6.311735,-0.373052,-0.566450,NaN,NaN,NaN,NaN,NaN
1,20 Microns Ltd.,11.0,20MICRONS,2008-10-07,15.05,-0.105232,0.023232,-0.669144,-0.645911,-1.502487,0.184699,NaN,NaN,NaN,NaN,NaN
2,20 Microns Ltd.,11.0,20MICRONS,2008-10-08,13.25,-0.119601,0.023232,-3.533362,-3.510130,-1.780674,0.072932,NaN,NaN,NaN,NaN,NaN
3,20 Microns Ltd.,11.0,20MICRONS,2008-10-10,11.60,-0.124528,0.045520,-7.052324,-7.006804,0.217126,0.354629,NaN,NaN,NaN,NaN,NaN
4,20 Microns Ltd.,11.0,20MICRONS,2008-10-13,12.32,0.062069,0.066863,5.042738,5.109602,-2.437097,0.145853,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13465088,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-21,0.35,0.000000,0.018215,1.441807,1.460021,0.572866,1.270303,NaN,NaN,NaN,NaN,NaN
13465089,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-22,0.35,0.000000,0.018215,0.593324,0.611538,0.752002,0.124527,NaN,NaN,NaN,NaN,NaN
13465090,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-26,0.35,0.000000,0.072878,-0.044514,0.028364,-1.091597,0.440605,NaN,NaN,NaN,NaN,NaN
13465091,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-27,0.35,0.000000,0.018215,0.326702,0.344916,-0.165072,-0.188505,NaN,NaN,NaN,NaN,NaN


In [5]:
# FUNCTION

# def CAR120(frame):
#     if ((~frame.OLS120_intercept.isnull().any()) == True):
#         ER = frame.RF + (frame.RMRF * frame.OLS120_RMRF) + (frame.SMB * frame.OLS120_SMB) + (frame.HML * frame.OLS120_HML) + frame.OLS120_intercept
#         AR = frame.pct - ER
#         return AR.sum()
#     else: 
#         return np.NaN

    # for i in range(len(frame)):
    #     if (i>0) & ( i< len(frame)-1) :
    #         coeff120CAR = np.array([frame.iloc[i]["OLS120_RMRF"], frame.iloc[i]["OLS120_SMB"],
    #                                 frame.iloc[i]["OLS120_HML"], 1, frame.iloc[i]["OLS120_intercept"]])

    #         event120CAR3 = np.array(frame.loc[ [i-1, i, i+1], ["RMRF", "SMB", "HML", "RF"]],)
    #         event120CAR3 = [np.append(item, 1) for item in event120CAR3]

    #         ER = np.matmul(event120CAR3, coeff120CAR)

    #         actualReturns = np.array([frame.iloc[i-1]["pct"], frame.iloc[i]["pct"], frame.iloc[i+1]["pct"]])

    #         AR = np.subtract( actualReturns, ER )

    #         output120CAR3.loc[i, ["120CAR3"]] = AR.sum()


    
        # if i>=2:
        #     frame["120CAR5"].iloc[i] = CAR120(frame.iloc[ i-2 : i+2 ])
        # if i>=3:
        #     frame["120CAR7"].iloc[i] = CAR120(frame.iloc[ i-3 : i+3 ])
        # if i>=5:
        #     frame["120CAR11"].iloc[i] = CAR120(frame.iloc[ i-5 : i+5 ])

        

def CAR(frame):

    outputFrame = pd.DataFrame( index = frame.index)

        
    # 3 Days
    
    outputFrame["pct_cen3sum"] = frame["pct"].rolling(window = 3, min_periods = 3, center = True).sum()
    
    outputFrame["RF_cen3sum"] = frame["RF"].rolling(window = 3, min_periods = 3, center = True).sum()
    
    outputFrame["RMRF_cen3sum"] = frame["RMRF"].rolling(window = 3, min_periods = 3, center = True).sum()
    
    # 5 Days
    
    outputFrame["pct_cen5sum"] = frame["pct"].rolling(window = 5, min_periods = 5, center = True).sum()
    
    outputFrame["RF_cen5sum"] = frame["RF"].rolling(window = 5, min_periods = 5, center = True).sum()
    
    outputFrame["RMRF_cen5sum"] = frame["RMRF"].rolling(window = 5, min_periods = 5, center = True).sum()
    
    
    # 7 Days
    
    outputFrame["pct_cen7sum"] = frame["pct"].rolling(window = 7, min_periods = 7, center = True).sum()
    
    outputFrame["RF_cen7sum"] = frame["RF"].rolling(window = 7, min_periods = 7, center = True).sum()
    
    outputFrame["RMRF_cen7sum"] = frame["RMRF"].rolling(window = 7, min_periods = 7, center = True).sum()
    
    
    # 11 Days
    
    outputFrame["pct_cen11sum"] = frame["pct"].rolling(window = 11, min_periods = 11, center = True).sum()
    
    outputFrame["RF_cen11sum"] = frame["RF"].rolling(window = 11, min_periods = 11, center = True).sum()
    
    outputFrame["RMRF_cen11sum"] = frame["RMRF"].rolling(window = 11, min_periods = 11, center = True).sum()
    
    
    return outputFrame


In [6]:
result120CAR = ols120CAR.sort_values(by = ["CompanyName", "AsOnDate"]).groupby(by = "CompanyName").progress_apply(CAR)

  0%|          | 0/3721 [00:00<?, ?it/s]

In [7]:
result120CAR

pct_cen3sum  RF_cen3sum  RMRF_cen3sum  \
CompanyName                                                          
20 Microns Ltd.    0                 NaN         NaN           NaN   
                   1                 NaN    0.116178    -10.583954   
                   2           -0.349361    0.091985    -11.254830   
                   3           -0.182061    0.135616     -5.542948   
                   4           -0.163920    0.134667     -0.834193   
...                                  ...         ...           ...   
Zylog Systems Ltd. 13465088     0.000000    0.054643      2.222872   
                   13465089     0.000000    0.109307      1.990617   
                   13465090     0.000000    0.109307      0.875511   
                   13465091     0.000000    0.109307      0.985242   
                   13465092          NaN         NaN           NaN   

                             pct_cen5sum  RF_cen5sum  RMRF_cen5sum  \
CompanyName                                                          
20 Microns Ltd.    0                 NaN         NaN           NaN   
                   1                 NaN         NaN           NaN   
                   2                 NaN    0.228562    -12.593540   
                   3           -0.388754    0.181132     -5.036699   
                   4           -0.183251    0.180182     -9.543586   
...                                  ...         ...           ...   
Zylog Systems Ltd. 13465088     0.000000    0.145734      0.669402   
                   13465089     0.000000    0.145735      2.505059   
                   13465090     0.000000    0.145736      3.020372   
                   13465091          NaN         NaN           NaN   
                   13465092          NaN         NaN           NaN   

                             pct_cen7sum  RF_cen7sum  RMRF_cen7sum  \
CompanyName                                                          
20 Microns Ltd.    0                 NaN         NaN           NaN   
                   1                 NaN         NaN           NaN   
                   2                 NaN         NaN           NaN   
                   3                 NaN    0.273128    -16.594178   
                   4           -0.163688    0.225697    -12.320480   
...                                  ...         ...           ...   
Zylog Systems Ltd. 13465088     0.000000    0.218599      1.086124   
                   13465089     0.000000    0.182163      1.699158   
                   13465090          NaN         NaN           NaN   
                   13465091          NaN         NaN           NaN   
                   13465092          NaN         NaN           NaN   

                             pct_cen11sum  RF_cen11sum  RMRF_cen11sum  
CompanyName                                                            
20 Microns Ltd.    0                  NaN          NaN            NaN  
                   1                  NaN          NaN            NaN  
                   2                  NaN          NaN            NaN  
                   3                  NaN          NaN            NaN  
                   4                  NaN          NaN            NaN  
...                                   ...          ...            ...  
Zylog Systems Ltd. 13465088           NaN          NaN            NaN  
                   13465089           NaN          NaN            NaN  
                   13465090           NaN          NaN            NaN  
                   13465091           NaN          NaN            NaN  
                   13465092           NaN          NaN            NaN  

[13465093 rows x 12 columns]

In [8]:
result120CAR2 = result120CAR.reset_index(level = 0).drop("CompanyName", axis = 1)
result120CAR2

,pct_cen3sum,RF_cen3sum,RMRF_cen3sum,pct_cen5sum,RF_cen5sum,RMRF_cen5sum,pct_cen7sum,RF_cen7sum,RMRF_cen7sum,pct_cen11sum,RF_cen11sum,RMRF_cen11sum
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,0.116178,-10.583954,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.349361,0.091985,-11.254830,NaN,0.228562,-12.593540,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.182061,0.135616,-5.542948,-0.388754,0.181132,-5.036699,NaN,0.273128,-16.594178,NaN,NaN,NaN
4,-0.163920,0.134667,-0.834193,-0.183251,0.180182,-9.543586,-0.163688,0.225697,-12.320480,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
13465088,0.000000,0.054643,2.222872,0.000000,0.145734,0.669402,0.000000,0.218599,1.086124,NaN,NaN,NaN
13465089,0.000000,0.109307,1.990617,0.000000,0.145735,2.505059,0.000000,0.182163,1.699158,NaN,NaN,NaN
13465090,0.000000,0.109307,0.875511,0.000000,0.145736,3.020372,NaN,NaN,NaN,NaN,NaN,NaN
13465091,0.000000,0.109307,0.985242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
output120CAR = pd.concat([ols120CAR, result120CAR2], axis = 1)

In [10]:
output120CAR.columns

Index(['CompanyName', 'ProwessCode', 'Symbol', 'AsOnDate', 'ACP', 'pct', 'RF',
       'RMRF', 'MF', 'SMB', 'HML', 'OLS120_intercept', 'OLS120_RMRF',
       'OLS120_r_squared', 'OLS120_adjusted_r_squared', 'OLS120_f_p_value',
       'pct_cen3sum', 'RF_cen3sum', 'RMRF_cen3sum', 'pct_cen5sum',
       'RF_cen5sum', 'RMRF_cen5sum', 'pct_cen7sum', 'RF_cen7sum',
       'RMRF_cen7sum', 'pct_cen11sum', 'RF_cen11sum', 'RMRF_cen11sum'],
      dtype='object')

In [11]:
output120CAR["120CAR3"] = output120CAR["pct_cen3sum"] - (output120CAR["RF_cen3sum"] + 
                          
                          output120CAR["OLS120_RMRF"]*output120CAR["RMRF_cen3sum"] +  
                          output120CAR["OLS120_intercept"]*3 )


In [12]:
output120CAR["120CAR5"] = output120CAR["pct_cen5sum"] - (output120CAR["RF_cen5sum"] + 
                          
                          output120CAR["OLS120_RMRF"]*output120CAR["RMRF_cen5sum"] + 
                          output120CAR["OLS120_intercept"]*3 )

In [13]:
output120CAR["120CAR7"] = output120CAR["pct_cen7sum"] - (output120CAR["RF_cen7sum"] + 
                          
                          output120CAR["OLS120_RMRF"]*output120CAR["RMRF_cen7sum"] + 
                          output120CAR["OLS120_intercept"]*3 )

In [14]:
output120CAR["120CAR11"] = output120CAR["pct_cen11sum"] - (output120CAR["RF_cen11sum"] + 
                          
                          output120CAR["OLS120_RMRF"]*output120CAR["RMRF_cen11sum"] + 
                          output120CAR["OLS120_intercept"]*3 )

In [15]:
output120CAR

,CompanyName,ProwessCode,Symbol,AsOnDate,ACP,pct,RF,RMRF,MF,SMB,HML,OLS120_intercept,OLS120_RMRF,OLS120_r_squared,OLS120_adjusted_r_squared,OLS120_f_p_value,pct_cen3sum,RF_cen3sum,RMRF_cen3sum,pct_cen5sum,RF_cen5sum,RMRF_cen5sum,pct_cen7sum,RF_cen7sum,RMRF_cen7sum,pct_cen11sum,RF_cen11sum,RMRF_cen11sum,120CAR3,120CAR5,120CAR7,120CAR11
0,20 Microns Ltd.,11.0,20MICRONS,2008-10-06,16.82,NaN,0.069713,-6.381449,-6.311735,-0.373052,-0.566450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20 Microns Ltd.,11.0,20MICRONS,2008-10-07,15.05,-0.105232,0.023232,-0.669144,-0.645911,-1.502487,0.184699,NaN,NaN,NaN,NaN,NaN,NaN,0.116178,-10.583954,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20 Microns Ltd.,11.0,20MICRONS,2008-10-08,13.25,-0.119601,0.023232,-3.533362,-3.510130,-1.780674,0.072932,NaN,NaN,NaN,NaN,NaN,-0.349361,0.091985,-11.254830,NaN,0.228562,-12.593540,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20 Microns Ltd.,11.0,20MICRONS,2008-10-10,11.60,-0.124528,0.045520,-7.052324,-7.006804,0.217126,0.354629,NaN,NaN,NaN,NaN,NaN,-0.182061,0.135616,-5.542948,-0.388754,0.181132,-5.036699,NaN,0.273128,-16.594178,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20 Microns Ltd.,11.0,20MICRONS,2008-10-13,12.32,0.062069,0.066863,5.042738,5.109602,-2.437097,0.145853,NaN,NaN,NaN,NaN,NaN,-0.163920,0.134667,-0.834193,-0.183251,0.180182,-9.543586,-0.163688,0.225697,-12.320480,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13465088,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-21,0.35,0.000000,0.018215,1.441807,1.460021,0.572866,1.270303,NaN,NaN,NaN,NaN,NaN,0.000000,0.054643,2.222872,0.000000,0.145734,0.669402,0.000000,0.218599,1.086124,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13465089,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-22,0.35,0.000000,0.018215,0.593324,0.611538,0.752002,0.124527,NaN,NaN,NaN,NaN,NaN,0.000000,0.109307,1.990617,0.000000,0.145735,2.505059,0.000000,0.182163,1.699158,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13465090,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-26,0.35,0.000000,0.072878,-0.044514,0.028364,-1.091597,0.440605,NaN,NaN,NaN,NaN,NaN,0.000000,0.109307,0.875511,0.000000,0.145736,3.020372,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13465091,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-27,0.35,0.000000,0.018215,0.326702,0.344916,-0.165072,-0.188505,NaN,NaN,NaN,NaN,NaN,0.000000,0.109307,0.985242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
output120CAR_2 = output120CAR.dropna(subset = "120CAR3").reset_index(drop = True).sort_values( by = ["CompanyName", "AsOnDate"] ).reset_index(drop = True)

In [17]:
output120CAR_2

,CompanyName,ProwessCode,Symbol,AsOnDate,ACP,pct,RF,RMRF,MF,SMB,HML,OLS120_intercept,OLS120_RMRF,OLS120_r_squared,OLS120_adjusted_r_squared,OLS120_f_p_value,pct_cen3sum,RF_cen3sum,RMRF_cen3sum,pct_cen5sum,RF_cen5sum,RMRF_cen5sum,pct_cen7sum,RF_cen7sum,RMRF_cen7sum,pct_cen11sum,RF_cen11sum,RMRF_cen11sum,120CAR3,120CAR5,120CAR7,120CAR11
0,20 Microns Ltd.,11.0,20MICRONS,2009-08-27,15.78,0.000000,0.009040,0.358959,0.368000,0.741064,0.945923,-0.012672,0.002374,0.016580,0.008245,1.610374e-01,-0.020051,0.027229,2.106517,0.095158,0.063718,2.095520,0.186179,0.099990,3.145210,0.367888,0.136368,5.257233,-0.014266,0.064480,0.116737,0.257054
1,20 Microns Ltd.,11.0,20MICRONS,2011-04-29,20.48,-0.016330,0.019851,-0.595940,-0.576089,-0.572908,-0.607305,-0.027709,0.010815,0.109751,0.102207,2.190040e-04,0.135960,0.099051,-2.055273,0.068979,0.138538,-4.268887,0.048651,0.178026,-4.607642,0.066696,0.316887,-5.076389,0.142262,0.059733,0.003581,-0.112165
2,20 Microns Ltd.,11.0,20MICRONS,2014-08-06,31.15,-0.011111,0.022705,-0.690542,-0.667837,1.023324,-0.182707,-0.034611,0.011129,0.088050,0.080322,9.984509e-04,-0.012716,0.068115,-0.250139,-0.004716,0.158951,-0.558310,-0.002971,0.249787,-1.450041,-0.040029,0.363107,-1.042710,0.025785,-0.053622,-0.132788,-0.287699
3,20 Microns Ltd.,11.0,20MICRONS,2017-05-04,43.70,0.016279,0.016554,0.046274,0.062828,0.831535,-0.581499,-0.024352,0.021487,0.307332,0.301462,5.063329e-11,-0.023826,0.049554,-0.994942,-0.031524,0.165027,-0.704433,-0.043378,0.198027,-0.312208,-0.078551,0.263922,0.266193,0.021052,-0.108360,-0.161642,-0.275138
4,20 Microns Ltd.,11.0,20MICRONS,2019-05-28,41.00,-0.016787,0.016873,0.367905,0.384778,-0.177790,0.292518,-0.026474,0.016340,0.208984,0.202280,1.531736e-07,0.060421,0.084375,0.593600,0.127139,0.117695,2.441371,0.116773,0.151016,2.141790,0.099791,0.250771,1.525870,0.045767,0.048972,0.010181,-0.096492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20518,Zylog Systems Ltd.,275793.0,ZYLOG,2014-02-14,11.40,0.004405,0.023865,0.579200,0.603064,-0.772576,0.455377,-0.038032,0.011159,0.057327,0.049338,8.443439e-03,-0.021532,0.119340,-0.605400,-0.056325,0.166753,-0.114553,-0.073061,0.214166,0.387515,-0.100032,0.356106,0.623586,-0.020020,-0.107702,-0.177455,-0.349000
20519,Zylog Systems Ltd.,275793.0,ZYLOG,2014-11-19,7.70,-0.025316,0.021860,-0.741569,-0.719709,0.280029,-0.546563,-0.037219,0.014717,0.059441,0.051470,7.287379e-03,-0.166621,0.065476,-0.584795,-0.128630,0.152826,0.041803,-0.054409,0.239965,0.676165,-0.144981,0.327195,-0.398142,-0.111834,-0.170415,-0.192669,-0.354660
20520,Zylog Systems Ltd.,275793.0,ZYLOG,2015-08-14,5.40,0.018868,0.019213,1.546309,1.565522,-0.064032,1.416502,-0.034858,0.007051,0.027495,0.019253,7.030236e-02,-0.113635,0.096076,0.535057,-0.246220,0.134926,-0.534668,-0.291712,0.173777,-0.844522,-0.274151,0.291613,-3.755096,-0.108911,-0.272803,-0.354960,-0.434712
20521,Zylog Systems Ltd.,275793.0,ZYLOG,2016-11-23,4.25,-0.022989,0.015807,1.118596,1.134402,0.640072,1.506896,-0.028364,0.006545,0.020030,0.011725,1.230994e-01,0.000961,0.047206,1.977908,-0.072004,0.110227,1.658702,-0.103254,0.172820,2.689463,0.087481,0.236579,3.498012,0.025903,-0.107993,-0.208582,-0.086900


In [18]:
output120CAR_2.to_pickle(rf"{output_folder_path}\output120CAR_2.pkl")

In [19]:
del ols120
del ols120CAR

del result120CAR2
del result120CAR

del output120CAR
del output120CAR_2

## 150 CAR Data Reading

In [20]:
ols150 = pd.read_pickle(rf"{import_folder_path}\ols150_2.pkl")

## 150 CAR Calc

In [21]:
ols150CAR = ols150.sort_values(by = ["CompanyName", "AsOnDate"])
ols150CAR

,CompanyName,ProwessCode,Symbol,AsOnDate,ACP,pct,RF,RMRF,MF,SMB,HML,OLS150_intercept,OLS150_RMRF,OLS150_r_squared,OLS150_adjusted_r_squared,OLS150_f_p_value
0,20 Microns Ltd.,11.0,20MICRONS,2008-10-06,16.82,NaN,0.069713,-6.381449,-6.311735,-0.373052,-0.566450,NaN,NaN,NaN,NaN,NaN
1,20 Microns Ltd.,11.0,20MICRONS,2008-10-07,15.05,-0.105232,0.023232,-0.669144,-0.645911,-1.502487,0.184699,NaN,NaN,NaN,NaN,NaN
2,20 Microns Ltd.,11.0,20MICRONS,2008-10-08,13.25,-0.119601,0.023232,-3.533362,-3.510130,-1.780674,0.072932,NaN,NaN,NaN,NaN,NaN
3,20 Microns Ltd.,11.0,20MICRONS,2008-10-10,11.60,-0.124528,0.045520,-7.052324,-7.006804,0.217126,0.354629,NaN,NaN,NaN,NaN,NaN
4,20 Microns Ltd.,11.0,20MICRONS,2008-10-13,12.32,0.062069,0.066863,5.042738,5.109602,-2.437097,0.145853,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13465088,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-21,0.35,0.000000,0.018215,1.441807,1.460021,0.572866,1.270303,NaN,NaN,NaN,NaN,NaN
13465089,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-22,0.35,0.000000,0.018215,0.593324,0.611538,0.752002,0.124527,NaN,NaN,NaN,NaN,NaN
13465090,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-26,0.35,0.000000,0.072878,-0.044514,0.028364,-1.091597,0.440605,NaN,NaN,NaN,NaN,NaN
13465091,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-27,0.35,0.000000,0.018215,0.326702,0.344916,-0.165072,-0.188505,NaN,NaN,NaN,NaN,NaN


In [22]:
# FUNCTION

# def CAR150(frame):
#     if ((~frame.OLS150_intercept.isnull().any()) == True):
#         ER = frame.RF + (frame.RMRF * frame.OLS150_RMRF) + (frame.SMB * frame.OLS150_SMB) + (frame.HML * frame.OLS150_HML) + frame.OLS150_intercept
#         AR = frame.pct - ER
#         return AR.sum()
#     else: 
#         return np.NaN

    # for i in range(len(frame)):
    #     if (i>0) & ( i< len(frame)-1) :
    #         coeff150CAR = np.array([frame.iloc[i]["OLS150_RMRF"], frame.iloc[i]["OLS150_SMB"],
    #                                 frame.iloc[i]["OLS150_HML"], 1, frame.iloc[i]["OLS150_intercept"]])

    #         event150CAR3 = np.array(frame.loc[ [i-1, i, i+1], ["RMRF", "SMB", "HML", "RF"]],)
    #         event150CAR3 = [np.append(item, 1) for item in event150CAR3]

    #         ER = np.matmul(event150CAR3, coeff150CAR)

    #         actualReturns = np.array([frame.iloc[i-1]["pct"], frame.iloc[i]["pct"], frame.iloc[i+1]["pct"]])

    #         AR = np.subtract( actualReturns, ER )

    #         output150CAR3.loc[i, ["150CAR3"]] = AR.sum()


    
        # if i>=2:
        #     frame["150CAR5"].iloc[i] = CAR150(frame.iloc[ i-2 : i+2 ])
        # if i>=3:
        #     frame["150CAR7"].iloc[i] = CAR150(frame.iloc[ i-3 : i+3 ])
        # if i>=5:
        #     frame["150CAR11"].iloc[i] = CAR150(frame.iloc[ i-5 : i+5 ])

        

def CAR(frame):

    outputFrame = pd.DataFrame( index = frame.index)

        
    # 3 Days
    
    outputFrame["pct_cen3sum"] = frame["pct"].rolling(window = 3, min_periods = 3, center = True).sum()
    
    outputFrame["RF_cen3sum"] = frame["RF"].rolling(window = 3, min_periods = 3, center = True).sum()
    
    outputFrame["RMRF_cen3sum"] = frame["RMRF"].rolling(window = 3, min_periods = 3, center = True).sum()
    
    
    # 5 Days
    
    outputFrame["pct_cen5sum"] = frame["pct"].rolling(window = 5, min_periods = 5, center = True).sum()
    
    outputFrame["RF_cen5sum"] = frame["RF"].rolling(window = 5, min_periods = 5, center = True).sum()
    
    outputFrame["RMRF_cen5sum"] = frame["RMRF"].rolling(window = 5, min_periods = 5, center = True).sum()
        
    
    # 7 Days
    
    outputFrame["pct_cen7sum"] = frame["pct"].rolling(window = 7, min_periods = 7, center = True).sum()
    
    outputFrame["RF_cen7sum"] = frame["RF"].rolling(window = 7, min_periods = 7, center = True).sum()
    
    outputFrame["RMRF_cen7sum"] = frame["RMRF"].rolling(window = 7, min_periods = 7, center = True).sum()
        
    
    # 11 Days
    
    outputFrame["pct_cen11sum"] = frame["pct"].rolling(window = 11, min_periods = 11, center = True).sum()
    
    outputFrame["RF_cen11sum"] = frame["RF"].rolling(window = 11, min_periods = 11, center = True).sum()
    
    outputFrame["RMRF_cen11sum"] = frame["RMRF"].rolling(window = 11, min_periods = 11, center = True).sum()
        
    return outputFrame


In [23]:
result150CAR = ols150CAR.sort_values(by = ["CompanyName", "AsOnDate"]).groupby(by = "CompanyName").progress_apply(CAR)

  0%|          | 0/3721 [00:00<?, ?it/s]

In [24]:
result150CAR

pct_cen3sum  RF_cen3sum  RMRF_cen3sum  \
CompanyName                                                          
20 Microns Ltd.    0                 NaN         NaN           NaN   
                   1                 NaN    0.116178    -10.583954   
                   2           -0.349361    0.091985    -11.254830   
                   3           -0.182061    0.135616     -5.542948   
                   4           -0.163920    0.134667     -0.834193   
...                                  ...         ...           ...   
Zylog Systems Ltd. 13465088     0.000000    0.054643      2.222872   
                   13465089     0.000000    0.109307      1.990617   
                   13465090     0.000000    0.109307      0.875511   
                   13465091     0.000000    0.109307      0.985242   
                   13465092          NaN         NaN           NaN   

                             pct_cen5sum  RF_cen5sum  RMRF_cen5sum  \
CompanyName                                                          
20 Microns Ltd.    0                 NaN         NaN           NaN   
                   1                 NaN         NaN           NaN   
                   2                 NaN    0.228562    -12.593540   
                   3           -0.388754    0.181132     -5.036699   
                   4           -0.183251    0.180182     -9.543586   
...                                  ...         ...           ...   
Zylog Systems Ltd. 13465088     0.000000    0.145734      0.669402   
                   13465089     0.000000    0.145735      2.505059   
                   13465090     0.000000    0.145736      3.020372   
                   13465091          NaN         NaN           NaN   
                   13465092          NaN         NaN           NaN   

                             pct_cen7sum  RF_cen7sum  RMRF_cen7sum  \
CompanyName                                                          
20 Microns Ltd.    0                 NaN         NaN           NaN   
                   1                 NaN         NaN           NaN   
                   2                 NaN         NaN           NaN   
                   3                 NaN    0.273128    -16.594178   
                   4           -0.163688    0.225697    -12.320480   
...                                  ...         ...           ...   
Zylog Systems Ltd. 13465088     0.000000    0.218599      1.086124   
                   13465089     0.000000    0.182163      1.699158   
                   13465090          NaN         NaN           NaN   
                   13465091          NaN         NaN           NaN   
                   13465092          NaN         NaN           NaN   

                             pct_cen11sum  RF_cen11sum  RMRF_cen11sum  
CompanyName                                                            
20 Microns Ltd.    0                  NaN          NaN            NaN  
                   1                  NaN          NaN            NaN  
                   2                  NaN          NaN            NaN  
                   3                  NaN          NaN            NaN  
                   4                  NaN          NaN            NaN  
...                                   ...          ...            ...  
Zylog Systems Ltd. 13465088           NaN          NaN            NaN  
                   13465089           NaN          NaN            NaN  
                   13465090           NaN          NaN            NaN  
                   13465091           NaN          NaN            NaN  
                   13465092           NaN          NaN            NaN  

[13465093 rows x 12 columns]

In [25]:
result150CAR2 = result150CAR.reset_index(level = 0).drop("CompanyName", axis = 1)
result150CAR2

,pct_cen3sum,RF_cen3sum,RMRF_cen3sum,pct_cen5sum,RF_cen5sum,RMRF_cen5sum,pct_cen7sum,RF_cen7sum,RMRF_cen7sum,pct_cen11sum,RF_cen11sum,RMRF_cen11sum
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,0.116178,-10.583954,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.349361,0.091985,-11.254830,NaN,0.228562,-12.593540,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.182061,0.135616,-5.542948,-0.388754,0.181132,-5.036699,NaN,0.273128,-16.594178,NaN,NaN,NaN
4,-0.163920,0.134667,-0.834193,-0.183251,0.180182,-9.543586,-0.163688,0.225697,-12.320480,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
13465088,0.000000,0.054643,2.222872,0.000000,0.145734,0.669402,0.000000,0.218599,1.086124,NaN,NaN,NaN
13465089,0.000000,0.109307,1.990617,0.000000,0.145735,2.505059,0.000000,0.182163,1.699158,NaN,NaN,NaN
13465090,0.000000,0.109307,0.875511,0.000000,0.145736,3.020372,NaN,NaN,NaN,NaN,NaN,NaN
13465091,0.000000,0.109307,0.985242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
output150CAR = pd.concat([ols150CAR, result150CAR2], axis = 1)
output150CAR

,CompanyName,ProwessCode,Symbol,AsOnDate,ACP,pct,RF,RMRF,MF,SMB,HML,OLS150_intercept,OLS150_RMRF,OLS150_r_squared,OLS150_adjusted_r_squared,OLS150_f_p_value,pct_cen3sum,RF_cen3sum,RMRF_cen3sum,pct_cen5sum,RF_cen5sum,RMRF_cen5sum,pct_cen7sum,RF_cen7sum,RMRF_cen7sum,pct_cen11sum,RF_cen11sum,RMRF_cen11sum
0,20 Microns Ltd.,11.0,20MICRONS,2008-10-06,16.82,NaN,0.069713,-6.381449,-6.311735,-0.373052,-0.566450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20 Microns Ltd.,11.0,20MICRONS,2008-10-07,15.05,-0.105232,0.023232,-0.669144,-0.645911,-1.502487,0.184699,NaN,NaN,NaN,NaN,NaN,NaN,0.116178,-10.583954,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20 Microns Ltd.,11.0,20MICRONS,2008-10-08,13.25,-0.119601,0.023232,-3.533362,-3.510130,-1.780674,0.072932,NaN,NaN,NaN,NaN,NaN,-0.349361,0.091985,-11.254830,NaN,0.228562,-12.593540,NaN,NaN,NaN,NaN,NaN,NaN
3,20 Microns Ltd.,11.0,20MICRONS,2008-10-10,11.60,-0.124528,0.045520,-7.052324,-7.006804,0.217126,0.354629,NaN,NaN,NaN,NaN,NaN,-0.182061,0.135616,-5.542948,-0.388754,0.181132,-5.036699,NaN,0.273128,-16.594178,NaN,NaN,NaN
4,20 Microns Ltd.,11.0,20MICRONS,2008-10-13,12.32,0.062069,0.066863,5.042738,5.109602,-2.437097,0.145853,NaN,NaN,NaN,NaN,NaN,-0.163920,0.134667,-0.834193,-0.183251,0.180182,-9.543586,-0.163688,0.225697,-12.320480,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13465088,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-21,0.35,0.000000,0.018215,1.441807,1.460021,0.572866,1.270303,NaN,NaN,NaN,NaN,NaN,0.000000,0.054643,2.222872,0.000000,0.145734,0.669402,0.000000,0.218599,1.086124,NaN,NaN,NaN
13465089,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-22,0.35,0.000000,0.018215,0.593324,0.611538,0.752002,0.124527,NaN,NaN,NaN,NaN,NaN,0.000000,0.109307,1.990617,0.000000,0.145735,2.505059,0.000000,0.182163,1.699158,NaN,NaN,NaN
13465090,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-26,0.35,0.000000,0.072878,-0.044514,0.028364,-1.091597,0.440605,NaN,NaN,NaN,NaN,NaN,0.000000,0.109307,0.875511,0.000000,0.145736,3.020372,NaN,NaN,NaN,NaN,NaN,NaN
13465091,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-27,0.35,0.000000,0.018215,0.326702,0.344916,-0.165072,-0.188505,NaN,NaN,NaN,NaN,NaN,0.000000,0.109307,0.985242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
output150CAR.columns

Index(['CompanyName', 'ProwessCode', 'Symbol', 'AsOnDate', 'ACP', 'pct', 'RF',
       'RMRF', 'MF', 'SMB', 'HML', 'OLS150_intercept', 'OLS150_RMRF',
       'OLS150_r_squared', 'OLS150_adjusted_r_squared', 'OLS150_f_p_value',
       'pct_cen3sum', 'RF_cen3sum', 'RMRF_cen3sum', 'pct_cen5sum',
       'RF_cen5sum', 'RMRF_cen5sum', 'pct_cen7sum', 'RF_cen7sum',
       'RMRF_cen7sum', 'pct_cen11sum', 'RF_cen11sum', 'RMRF_cen11sum'],
      dtype='object')

In [28]:
output150CAR["150CAR3"] = output150CAR["pct_cen3sum"] - (output150CAR["RF_cen3sum"] + 
                          
                          output150CAR["OLS150_RMRF"]*output150CAR["RMRF_cen3sum"] + 
                          output150CAR["OLS150_intercept"]*3 )

In [29]:
output150CAR["150CAR5"] = output150CAR["pct_cen5sum"] - (output150CAR["RF_cen5sum"] + 
                          
                          output150CAR["OLS150_RMRF"]*output150CAR["RMRF_cen5sum"] + 
                          output150CAR["OLS150_intercept"]*3 )

In [30]:
output150CAR["150CAR7"] = output150CAR["pct_cen7sum"] - (output150CAR["RF_cen7sum"] + 
                          
                          output150CAR["OLS150_RMRF"]*output150CAR["RMRF_cen7sum"] + 
                          output150CAR["OLS150_intercept"]*3 )

In [31]:
output150CAR["150CAR11"] = output150CAR["pct_cen11sum"] - (output150CAR["RF_cen11sum"] + 
                          
                          output150CAR["OLS150_RMRF"]*output150CAR["RMRF_cen11sum"] + 
                          output150CAR["OLS150_intercept"]*3 )

In [32]:
output150CAR

,CompanyName,ProwessCode,Symbol,AsOnDate,ACP,pct,RF,RMRF,MF,SMB,HML,OLS150_intercept,OLS150_RMRF,OLS150_r_squared,OLS150_adjusted_r_squared,OLS150_f_p_value,pct_cen3sum,RF_cen3sum,RMRF_cen3sum,pct_cen5sum,RF_cen5sum,RMRF_cen5sum,pct_cen7sum,RF_cen7sum,RMRF_cen7sum,pct_cen11sum,RF_cen11sum,RMRF_cen11sum,150CAR3,150CAR5,150CAR7,150CAR11
0,20 Microns Ltd.,11.0,20MICRONS,2008-10-06,16.82,NaN,0.069713,-6.381449,-6.311735,-0.373052,-0.566450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20 Microns Ltd.,11.0,20MICRONS,2008-10-07,15.05,-0.105232,0.023232,-0.669144,-0.645911,-1.502487,0.184699,NaN,NaN,NaN,NaN,NaN,NaN,0.116178,-10.583954,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20 Microns Ltd.,11.0,20MICRONS,2008-10-08,13.25,-0.119601,0.023232,-3.533362,-3.510130,-1.780674,0.072932,NaN,NaN,NaN,NaN,NaN,-0.349361,0.091985,-11.254830,NaN,0.228562,-12.593540,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20 Microns Ltd.,11.0,20MICRONS,2008-10-10,11.60,-0.124528,0.045520,-7.052324,-7.006804,0.217126,0.354629,NaN,NaN,NaN,NaN,NaN,-0.182061,0.135616,-5.542948,-0.388754,0.181132,-5.036699,NaN,0.273128,-16.594178,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20 Microns Ltd.,11.0,20MICRONS,2008-10-13,12.32,0.062069,0.066863,5.042738,5.109602,-2.437097,0.145853,NaN,NaN,NaN,NaN,NaN,-0.163920,0.134667,-0.834193,-0.183251,0.180182,-9.543586,-0.163688,0.225697,-12.320480,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13465088,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-21,0.35,0.000000,0.018215,1.441807,1.460021,0.572866,1.270303,NaN,NaN,NaN,NaN,NaN,0.000000,0.054643,2.222872,0.000000,0.145734,0.669402,0.000000,0.218599,1.086124,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13465089,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-22,0.35,0.000000,0.018215,0.593324,0.611538,0.752002,0.124527,NaN,NaN,NaN,NaN,NaN,0.000000,0.109307,1.990617,0.000000,0.145735,2.505059,0.000000,0.182163,1.699158,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13465090,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-26,0.35,0.000000,0.072878,-0.044514,0.028364,-1.091597,0.440605,NaN,NaN,NaN,NaN,NaN,0.000000,0.109307,0.875511,0.000000,0.145736,3.020372,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13465091,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-27,0.35,0.000000,0.018215,0.326702,0.344916,-0.165072,-0.188505,NaN,NaN,NaN,NaN,NaN,0.000000,0.109307,0.985242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
output150CAR_2 = output150CAR.dropna(subset = "150CAR3").reset_index(drop = True)
output150CAR_2

,CompanyName,ProwessCode,Symbol,AsOnDate,ACP,pct,RF,RMRF,MF,SMB,HML,OLS150_intercept,OLS150_RMRF,OLS150_r_squared,OLS150_adjusted_r_squared,OLS150_f_p_value,pct_cen3sum,RF_cen3sum,RMRF_cen3sum,pct_cen5sum,RF_cen5sum,RMRF_cen5sum,pct_cen7sum,RF_cen7sum,RMRF_cen7sum,pct_cen11sum,RF_cen11sum,RMRF_cen11sum,150CAR3,150CAR5,150CAR7,150CAR11
0,20 Microns Ltd.,11.0,20MICRONS,2009-08-27,15.78,0.000000,0.009040,0.358959,0.368000,0.741064,0.945923,-0.016317,0.004135,0.048609,0.042181,6.705688e-03,-0.020051,0.027229,2.106517,0.095158,0.063718,2.095520,0.186179,0.099990,3.145210,0.367888,0.136368,5.257233,-0.007040,0.071726,0.122134,0.258731
1,20 Microns Ltd.,11.0,20MICRONS,2011-04-29,20.48,-0.016330,0.019851,-0.595940,-0.576089,-0.572908,-0.607305,-0.026340,0.010345,0.100758,0.094682,7.558843e-05,0.135960,0.099051,-2.055273,0.068979,0.138538,-4.268887,0.048651,0.178026,-4.607642,0.066696,0.316887,-5.076389,0.137190,0.053622,-0.002689,-0.118655
2,20 Microns Ltd.,11.0,20MICRONS,2014-08-06,31.15,-0.011111,0.022705,-0.690542,-0.667837,1.023324,-0.182707,-0.034378,0.008381,0.050199,0.043781,5.847399e-03,-0.012716,0.068115,-0.250139,-0.004716,0.158951,-0.558310,-0.002971,0.249787,-1.450041,-0.040029,0.363107,-1.042710,0.024399,-0.055854,-0.137471,-0.291262
3,20 Microns Ltd.,11.0,20MICRONS,2017-05-04,43.70,0.016279,0.016554,0.046274,0.062828,0.831535,-0.581499,-0.023276,0.020482,0.218318,0.213037,1.663112e-09,-0.023826,0.049554,-0.994942,-0.031524,0.165027,-0.704433,-0.043378,0.198027,-0.312208,-0.078551,0.263922,0.266193,0.016825,-0.112295,-0.165183,-0.278097
4,20 Microns Ltd.,11.0,20MICRONS,2019-05-28,41.00,-0.016787,0.016873,0.367905,0.384778,-0.177790,0.292518,-0.025899,0.016770,0.245558,0.240460,1.139830e-10,0.060421,0.084375,0.593600,0.127139,0.117695,2.441371,0.116773,0.151016,2.141790,0.099791,0.250771,1.525870,0.043788,0.046198,0.007535,-0.098872
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20389,Zylog Systems Ltd.,275793.0,ZYLOG,2014-02-14,11.40,0.004405,0.023865,0.579200,0.603064,-0.772576,0.455377,-0.042947,0.011107,0.060269,0.053920,2.461387e-03,-0.021532,0.119340,-0.605400,-0.056325,0.166753,-0.114553,-0.073061,0.214166,0.387515,-0.100032,0.356106,0.623586,-0.005308,-0.092966,-0.162692,-0.334225
20390,Zylog Systems Ltd.,275793.0,ZYLOG,2014-11-19,7.70,-0.025316,0.021860,-0.741569,-0.719709,0.280029,-0.546563,-0.038134,0.016314,0.070138,0.063855,1.056219e-03,-0.166621,0.065476,-0.584795,-0.128630,0.152826,0.041803,-0.054409,0.239965,0.676165,-0.144981,0.327195,-0.398142,-0.108154,-0.167737,-0.191004,-0.351279
20391,Zylog Systems Ltd.,275793.0,ZYLOG,2015-08-14,5.40,0.018868,0.019213,1.546309,1.565522,-0.064032,1.416502,-0.034571,0.010030,0.061262,0.054919,2.260515e-03,-0.113635,0.096076,0.535057,-0.246220,0.134926,-0.534668,-0.291712,0.173777,-0.844522,-0.274151,0.291613,-3.755096,-0.111364,-0.272070,-0.353304,-0.424385
20392,Zylog Systems Ltd.,275793.0,ZYLOG,2016-11-23,4.25,-0.022989,0.015807,1.118596,1.134402,0.640072,1.506896,-0.029032,0.004966,0.013618,0.006953,1.549859e-01,0.000961,0.047206,1.977908,-0.072004,0.110227,1.658702,-0.103254,0.172820,2.689463,0.087481,0.236579,3.498012,0.031027,-0.103373,-0.202336,-0.079376


In [34]:
output150CAR_2.to_pickle(rf"{output_folder_path}\output150CAR_2.pkl")

In [35]:
del ols150
del ols150CAR

del result150CAR2
del result150CAR

del output150CAR
del output150CAR_2

## 180 CAR Data Reading

In [36]:
ols180 = pd.read_pickle(rf"{import_folder_path}\ols180_2.pkl")

## 180 CAR Calc

In [37]:
ols180CAR = ols180.sort_values(by = ["CompanyName", "AsOnDate"])
ols180CAR

,CompanyName,ProwessCode,Symbol,AsOnDate,ACP,pct,RF,RMRF,MF,SMB,HML,OLS180_intercept,OLS180_RMRF,OLS180_r_squared,OLS180_adjusted_r_squared,OLS180_f_p_value
0,20 Microns Ltd.,11.0,20MICRONS,2008-10-06,16.82,NaN,0.069713,-6.381449,-6.311735,-0.373052,-0.566450,NaN,NaN,NaN,NaN,NaN
1,20 Microns Ltd.,11.0,20MICRONS,2008-10-07,15.05,-0.105232,0.023232,-0.669144,-0.645911,-1.502487,0.184699,NaN,NaN,NaN,NaN,NaN
2,20 Microns Ltd.,11.0,20MICRONS,2008-10-08,13.25,-0.119601,0.023232,-3.533362,-3.510130,-1.780674,0.072932,NaN,NaN,NaN,NaN,NaN
3,20 Microns Ltd.,11.0,20MICRONS,2008-10-10,11.60,-0.124528,0.045520,-7.052324,-7.006804,0.217126,0.354629,NaN,NaN,NaN,NaN,NaN
4,20 Microns Ltd.,11.0,20MICRONS,2008-10-13,12.32,0.062069,0.066863,5.042738,5.109602,-2.437097,0.145853,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13465088,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-21,0.35,0.000000,0.018215,1.441807,1.460021,0.572866,1.270303,NaN,NaN,NaN,NaN,NaN
13465089,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-22,0.35,0.000000,0.018215,0.593324,0.611538,0.752002,0.124527,NaN,NaN,NaN,NaN,NaN
13465090,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-26,0.35,0.000000,0.072878,-0.044514,0.028364,-1.091597,0.440605,NaN,NaN,NaN,NaN,NaN
13465091,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-27,0.35,0.000000,0.018215,0.326702,0.344916,-0.165072,-0.188505,NaN,NaN,NaN,NaN,NaN


In [38]:
# FUNCTION

# def CAR180(frame):
#     if ((~frame.OLS180_intercept.isnull().any()) == True):
#         ER = frame.RF + (frame.RMRF * frame.OLS180_RMRF) + (frame.SMB * frame.OLS180_SMB) + (frame.HML * frame.OLS180_HML) + frame.OLS180_intercept
#         AR = frame.pct - ER
#         return AR.sum()
#     else: 
#         return np.NaN

    # for i in range(len(frame)):
    #     if (i>0) & ( i< len(frame)-1) :
    #         coeff180CAR = np.array([frame.iloc[i]["OLS180_RMRF"], frame.iloc[i]["OLS180_SMB"],
    #                                 frame.iloc[i]["OLS180_HML"], 1, frame.iloc[i]["OLS180_intercept"]])

    #         event180CAR3 = np.array(frame.loc[ [i-1, i, i+1], ["RMRF", "SMB", "HML", "RF"]],)
    #         event180CAR3 = [np.append(item, 1) for item in event180CAR3]

    #         ER = np.matmul(event180CAR3, coeff180CAR)

    #         actualReturns = np.array([frame.iloc[i-1]["pct"], frame.iloc[i]["pct"], frame.iloc[i+1]["pct"]])

    #         AR = np.subtract( actualReturns, ER )

    #         output180CAR3.loc[i, ["180CAR3"]] = AR.sum()


    
        # if i>=2:
        #     frame["180CAR5"].iloc[i] = CAR180(frame.iloc[ i-2 : i+2 ])
        # if i>=3:
        #     frame["180CAR7"].iloc[i] = CAR180(frame.iloc[ i-3 : i+3 ])
        # if i>=5:
        #     frame["180CAR11"].iloc[i] = CAR180(frame.iloc[ i-5 : i+5 ])

        

def CAR(frame):

    outputFrame = pd.DataFrame( index = frame.index)

        
    # 3 Days
    
    outputFrame["pct_cen3sum"] = frame["pct"].rolling(window = 3, min_periods = 3, center = True).sum()
    
    outputFrame["RF_cen3sum"] = frame["RF"].rolling(window = 3, min_periods = 3, center = True).sum()
    
    outputFrame["RMRF_cen3sum"] = frame["RMRF"].rolling(window = 3, min_periods = 3, center = True).sum()
    
    
    # 5 Days
    
    outputFrame["pct_cen5sum"] = frame["pct"].rolling(window = 5, min_periods = 5, center = True).sum()
    
    outputFrame["RF_cen5sum"] = frame["RF"].rolling(window = 5, min_periods = 5, center = True).sum()
    
    outputFrame["RMRF_cen5sum"] = frame["RMRF"].rolling(window = 5, min_periods = 5, center = True).sum()
    
    
    # 7 Days
    
    outputFrame["pct_cen7sum"] = frame["pct"].rolling(window = 7, min_periods = 7, center = True).sum()
    
    outputFrame["RF_cen7sum"] = frame["RF"].rolling(window = 7, min_periods = 7, center = True).sum()
    
    outputFrame["RMRF_cen7sum"] = frame["RMRF"].rolling(window = 7, min_periods = 7, center = True).sum()
    
    
    # 11 Days
    
    outputFrame["pct_cen11sum"] = frame["pct"].rolling(window = 11, min_periods = 11, center = True).sum()
    
    outputFrame["RF_cen11sum"] = frame["RF"].rolling(window = 11, min_periods = 11, center = True).sum()
    
    outputFrame["RMRF_cen11sum"] = frame["RMRF"].rolling(window = 11, min_periods = 11, center = True).sum()
    
    
    return outputFrame


In [39]:
result180CAR = ols180CAR.sort_values(by = ["CompanyName", "AsOnDate"]).groupby(by = "CompanyName").progress_apply(CAR)

  0%|          | 0/3721 [00:00<?, ?it/s]

In [40]:
result180CAR

pct_cen3sum  RF_cen3sum  RMRF_cen3sum  \
CompanyName                                                          
20 Microns Ltd.    0                 NaN         NaN           NaN   
                   1                 NaN    0.116178    -10.583954   
                   2           -0.349361    0.091985    -11.254830   
                   3           -0.182061    0.135616     -5.542948   
                   4           -0.163920    0.134667     -0.834193   
...                                  ...         ...           ...   
Zylog Systems Ltd. 13465088     0.000000    0.054643      2.222872   
                   13465089     0.000000    0.109307      1.990617   
                   13465090     0.000000    0.109307      0.875511   
                   13465091     0.000000    0.109307      0.985242   
                   13465092          NaN         NaN           NaN   

                             pct_cen5sum  RF_cen5sum  RMRF_cen5sum  \
CompanyName                                                          
20 Microns Ltd.    0                 NaN         NaN           NaN   
                   1                 NaN         NaN           NaN   
                   2                 NaN    0.228562    -12.593540   
                   3           -0.388754    0.181132     -5.036699   
                   4           -0.183251    0.180182     -9.543586   
...                                  ...         ...           ...   
Zylog Systems Ltd. 13465088     0.000000    0.145734      0.669402   
                   13465089     0.000000    0.145735      2.505059   
                   13465090     0.000000    0.145736      3.020372   
                   13465091          NaN         NaN           NaN   
                   13465092          NaN         NaN           NaN   

                             pct_cen7sum  RF_cen7sum  RMRF_cen7sum  \
CompanyName                                                          
20 Microns Ltd.    0                 NaN         NaN           NaN   
                   1                 NaN         NaN           NaN   
                   2                 NaN         NaN           NaN   
                   3                 NaN    0.273128    -16.594178   
                   4           -0.163688    0.225697    -12.320480   
...                                  ...         ...           ...   
Zylog Systems Ltd. 13465088     0.000000    0.218599      1.086124   
                   13465089     0.000000    0.182163      1.699158   
                   13465090          NaN         NaN           NaN   
                   13465091          NaN         NaN           NaN   
                   13465092          NaN         NaN           NaN   

                             pct_cen11sum  RF_cen11sum  RMRF_cen11sum  
CompanyName                                                            
20 Microns Ltd.    0                  NaN          NaN            NaN  
                   1                  NaN          NaN            NaN  
                   2                  NaN          NaN            NaN  
                   3                  NaN          NaN            NaN  
                   4                  NaN          NaN            NaN  
...                                   ...          ...            ...  
Zylog Systems Ltd. 13465088           NaN          NaN            NaN  
                   13465089           NaN          NaN            NaN  
                   13465090           NaN          NaN            NaN  
                   13465091           NaN          NaN            NaN  
                   13465092           NaN          NaN            NaN  

[13465093 rows x 12 columns]

In [41]:
result180CAR2 = result180CAR.reset_index(level = 0).drop("CompanyName", axis = 1)
result180CAR2

,pct_cen3sum,RF_cen3sum,RMRF_cen3sum,pct_cen5sum,RF_cen5sum,RMRF_cen5sum,pct_cen7sum,RF_cen7sum,RMRF_cen7sum,pct_cen11sum,RF_cen11sum,RMRF_cen11sum
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,0.116178,-10.583954,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.349361,0.091985,-11.254830,NaN,0.228562,-12.593540,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.182061,0.135616,-5.542948,-0.388754,0.181132,-5.036699,NaN,0.273128,-16.594178,NaN,NaN,NaN
4,-0.163920,0.134667,-0.834193,-0.183251,0.180182,-9.543586,-0.163688,0.225697,-12.320480,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
13465088,0.000000,0.054643,2.222872,0.000000,0.145734,0.669402,0.000000,0.218599,1.086124,NaN,NaN,NaN
13465089,0.000000,0.109307,1.990617,0.000000,0.145735,2.505059,0.000000,0.182163,1.699158,NaN,NaN,NaN
13465090,0.000000,0.109307,0.875511,0.000000,0.145736,3.020372,NaN,NaN,NaN,NaN,NaN,NaN
13465091,0.000000,0.109307,0.985242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
output180CAR = pd.concat([ols180CAR, result180CAR2], axis = 1)
output180CAR

,CompanyName,ProwessCode,Symbol,AsOnDate,ACP,pct,RF,RMRF,MF,SMB,HML,OLS180_intercept,OLS180_RMRF,OLS180_r_squared,OLS180_adjusted_r_squared,OLS180_f_p_value,pct_cen3sum,RF_cen3sum,RMRF_cen3sum,pct_cen5sum,RF_cen5sum,RMRF_cen5sum,pct_cen7sum,RF_cen7sum,RMRF_cen7sum,pct_cen11sum,RF_cen11sum,RMRF_cen11sum
0,20 Microns Ltd.,11.0,20MICRONS,2008-10-06,16.82,NaN,0.069713,-6.381449,-6.311735,-0.373052,-0.566450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20 Microns Ltd.,11.0,20MICRONS,2008-10-07,15.05,-0.105232,0.023232,-0.669144,-0.645911,-1.502487,0.184699,NaN,NaN,NaN,NaN,NaN,NaN,0.116178,-10.583954,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20 Microns Ltd.,11.0,20MICRONS,2008-10-08,13.25,-0.119601,0.023232,-3.533362,-3.510130,-1.780674,0.072932,NaN,NaN,NaN,NaN,NaN,-0.349361,0.091985,-11.254830,NaN,0.228562,-12.593540,NaN,NaN,NaN,NaN,NaN,NaN
3,20 Microns Ltd.,11.0,20MICRONS,2008-10-10,11.60,-0.124528,0.045520,-7.052324,-7.006804,0.217126,0.354629,NaN,NaN,NaN,NaN,NaN,-0.182061,0.135616,-5.542948,-0.388754,0.181132,-5.036699,NaN,0.273128,-16.594178,NaN,NaN,NaN
4,20 Microns Ltd.,11.0,20MICRONS,2008-10-13,12.32,0.062069,0.066863,5.042738,5.109602,-2.437097,0.145853,NaN,NaN,NaN,NaN,NaN,-0.163920,0.134667,-0.834193,-0.183251,0.180182,-9.543586,-0.163688,0.225697,-12.320480,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13465088,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-21,0.35,0.000000,0.018215,1.441807,1.460021,0.572866,1.270303,NaN,NaN,NaN,NaN,NaN,0.000000,0.054643,2.222872,0.000000,0.145734,0.669402,0.000000,0.218599,1.086124,NaN,NaN,NaN
13465089,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-22,0.35,0.000000,0.018215,0.593324,0.611538,0.752002,0.124527,NaN,NaN,NaN,NaN,NaN,0.000000,0.109307,1.990617,0.000000,0.145735,2.505059,0.000000,0.182163,1.699158,NaN,NaN,NaN
13465090,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-26,0.35,0.000000,0.072878,-0.044514,0.028364,-1.091597,0.440605,NaN,NaN,NaN,NaN,NaN,0.000000,0.109307,0.875511,0.000000,0.145736,3.020372,NaN,NaN,NaN,NaN,NaN,NaN
13465091,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-27,0.35,0.000000,0.018215,0.326702,0.344916,-0.165072,-0.188505,NaN,NaN,NaN,NaN,NaN,0.000000,0.109307,0.985242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
output180CAR.columns

Index(['CompanyName', 'ProwessCode', 'Symbol', 'AsOnDate', 'ACP', 'pct', 'RF',
       'RMRF', 'MF', 'SMB', 'HML', 'OLS180_intercept', 'OLS180_RMRF',
       'OLS180_r_squared', 'OLS180_adjusted_r_squared', 'OLS180_f_p_value',
       'pct_cen3sum', 'RF_cen3sum', 'RMRF_cen3sum', 'pct_cen5sum',
       'RF_cen5sum', 'RMRF_cen5sum', 'pct_cen7sum', 'RF_cen7sum',
       'RMRF_cen7sum', 'pct_cen11sum', 'RF_cen11sum', 'RMRF_cen11sum'],
      dtype='object')

In [44]:
output180CAR["180CAR3"] = output180CAR["pct_cen3sum"] - (output180CAR["RF_cen3sum"] + 
                          
                          output180CAR["OLS180_RMRF"]*output180CAR["RMRF_cen3sum"] + 
                          output180CAR["OLS180_intercept"]*3 )


In [45]:
output180CAR["180CAR5"] = output180CAR["pct_cen5sum"] - (output180CAR["RF_cen5sum"] + 
                          
                          output180CAR["OLS180_RMRF"]*output180CAR["RMRF_cen5sum"] + 
                          output180CAR["OLS180_intercept"]*3 )

In [46]:
output180CAR["180CAR7"] = output180CAR["pct_cen7sum"] - (output180CAR["RF_cen7sum"] + 
                          
                          output180CAR["OLS180_RMRF"]*output180CAR["RMRF_cen7sum"] + 
                          output180CAR["OLS180_intercept"]*3 )

In [47]:
output180CAR["180CAR11"] = output180CAR["pct_cen11sum"] - (output180CAR["RF_cen11sum"] + 
                          
                          output180CAR["OLS180_RMRF"]*output180CAR["RMRF_cen11sum"] + 
                          output180CAR["OLS180_intercept"]*3 )

In [48]:
output180CAR

,CompanyName,ProwessCode,Symbol,AsOnDate,ACP,pct,RF,RMRF,MF,SMB,HML,OLS180_intercept,OLS180_RMRF,OLS180_r_squared,OLS180_adjusted_r_squared,OLS180_f_p_value,pct_cen3sum,RF_cen3sum,RMRF_cen3sum,pct_cen5sum,RF_cen5sum,RMRF_cen5sum,pct_cen7sum,RF_cen7sum,RMRF_cen7sum,pct_cen11sum,RF_cen11sum,RMRF_cen11sum,180CAR3,180CAR5,180CAR7,180CAR11
0,20 Microns Ltd.,11.0,20MICRONS,2008-10-06,16.82,NaN,0.069713,-6.381449,-6.311735,-0.373052,-0.566450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20 Microns Ltd.,11.0,20MICRONS,2008-10-07,15.05,-0.105232,0.023232,-0.669144,-0.645911,-1.502487,0.184699,NaN,NaN,NaN,NaN,NaN,NaN,0.116178,-10.583954,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20 Microns Ltd.,11.0,20MICRONS,2008-10-08,13.25,-0.119601,0.023232,-3.533362,-3.510130,-1.780674,0.072932,NaN,NaN,NaN,NaN,NaN,-0.349361,0.091985,-11.254830,NaN,0.228562,-12.593540,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20 Microns Ltd.,11.0,20MICRONS,2008-10-10,11.60,-0.124528,0.045520,-7.052324,-7.006804,0.217126,0.354629,NaN,NaN,NaN,NaN,NaN,-0.182061,0.135616,-5.542948,-0.388754,0.181132,-5.036699,NaN,0.273128,-16.594178,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20 Microns Ltd.,11.0,20MICRONS,2008-10-13,12.32,0.062069,0.066863,5.042738,5.109602,-2.437097,0.145853,NaN,NaN,NaN,NaN,NaN,-0.163920,0.134667,-0.834193,-0.183251,0.180182,-9.543586,-0.163688,0.225697,-12.320480,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13465088,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-21,0.35,0.000000,0.018215,1.441807,1.460021,0.572866,1.270303,NaN,NaN,NaN,NaN,NaN,0.000000,0.054643,2.222872,0.000000,0.145734,0.669402,0.000000,0.218599,1.086124,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13465089,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-22,0.35,0.000000,0.018215,0.593324,0.611538,0.752002,0.124527,NaN,NaN,NaN,NaN,NaN,0.000000,0.109307,1.990617,0.000000,0.145735,2.505059,0.000000,0.182163,1.699158,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13465090,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-26,0.35,0.000000,0.072878,-0.044514,0.028364,-1.091597,0.440605,NaN,NaN,NaN,NaN,NaN,0.000000,0.109307,0.875511,0.000000,0.145736,3.020372,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13465091,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-27,0.35,0.000000,0.018215,0.326702,0.344916,-0.165072,-0.188505,NaN,NaN,NaN,NaN,NaN,0.000000,0.109307,0.985242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
output180CAR_2 = output180CAR.dropna(subset = "180CAR3").reset_index(drop = True)
output180CAR_2

,CompanyName,ProwessCode,Symbol,AsOnDate,ACP,pct,RF,RMRF,MF,SMB,HML,OLS180_intercept,OLS180_RMRF,OLS180_r_squared,OLS180_adjusted_r_squared,OLS180_f_p_value,pct_cen3sum,RF_cen3sum,RMRF_cen3sum,pct_cen5sum,RF_cen5sum,RMRF_cen5sum,pct_cen7sum,RF_cen7sum,RMRF_cen7sum,pct_cen11sum,RF_cen11sum,RMRF_cen11sum,180CAR3,180CAR5,180CAR7,180CAR11
0,20 Microns Ltd.,11.0,20MICRONS,2009-08-27,15.78,0.000000,0.009040,0.358959,0.368000,0.741064,0.945923,-0.019785,0.006153,0.095433,0.090351,2.449373e-05,-0.020051,0.027229,2.106517,0.095158,0.063718,2.095520,0.186179,0.099990,3.145210,0.367888,0.136368,5.257233,-0.000888,0.077900,0.126189,0.258523
1,20 Microns Ltd.,11.0,20MICRONS,2011-04-29,20.48,-0.016330,0.019851,-0.595940,-0.576089,-0.572908,-0.607305,-0.024022,0.010026,0.072412,0.067200,2.594653e-04,0.135960,0.099051,-2.055273,0.068979,0.138538,-4.268887,0.048651,0.178026,-4.607642,0.066696,0.316887,-5.076389,0.129581,0.045305,-0.011114,-0.127230
2,20 Microns Ltd.,11.0,20MICRONS,2014-08-06,31.15,-0.011111,0.022705,-0.690542,-0.667837,1.023324,-0.182707,-0.034278,0.008417,0.042347,0.036967,5.581963e-03,-0.012716,0.068115,-0.250139,-0.004716,0.158951,-0.558310,-0.002971,0.249787,-1.450041,-0.040029,0.363107,-1.042710,0.024108,-0.056134,-0.137719,-0.291525
3,20 Microns Ltd.,11.0,20MICRONS,2017-05-04,43.70,0.016279,0.016554,0.046274,0.062828,0.831535,-0.581499,-0.024020,0.018310,0.186137,0.181564,1.480156e-09,-0.023826,0.049554,-0.994942,-0.031524,0.165027,-0.704433,-0.043378,0.198027,-0.312208,-0.078551,0.263922,0.266193,0.016897,-0.111592,-0.163628,-0.275287
4,20 Microns Ltd.,11.0,20MICRONS,2019-05-28,41.00,-0.016787,0.016873,0.367905,0.384778,-0.177790,0.292518,-0.024960,0.017704,0.224968,0.220614,1.744082e-11,0.060421,0.084375,0.593600,0.127139,0.117695,2.441371,0.116773,0.151016,2.141790,0.099791,0.250771,1.525870,0.040416,0.041101,0.002718,-0.103115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20143,Zylog Systems Ltd.,275793.0,ZYLOG,2014-02-14,11.40,0.004405,0.023865,0.579200,0.603064,-0.772576,0.455377,-0.040824,0.011775,0.065707,0.060458,5.140459e-04,-0.021532,0.119340,-0.605400,-0.056325,0.166753,-0.114553,-0.073061,0.214166,0.387515,-0.100032,0.356106,0.623586,-0.011272,-0.099257,-0.169318,-0.341008
20144,Zylog Systems Ltd.,275793.0,ZYLOG,2014-11-19,7.70,-0.025316,0.021860,-0.741569,-0.719709,0.280029,-0.546563,-0.037386,0.015785,0.067294,0.062054,4.372793e-04,-0.166621,0.065476,-0.584795,-0.128630,0.152826,0.041803,-0.054409,0.239965,0.676165,-0.144981,0.327195,-0.398142,-0.110707,-0.169957,-0.192889,-0.353733
20145,Zylog Systems Ltd.,275793.0,ZYLOG,2015-08-14,5.40,0.018868,0.019213,1.546309,1.565522,-0.064032,1.416502,-0.034362,0.011860,0.071526,0.066310,2.840090e-04,-0.113635,0.096076,0.535057,-0.246220,0.134926,-0.534668,-0.291712,0.173777,-0.844522,-0.274151,0.291613,-3.755096,-0.112971,-0.271720,-0.352387,-0.418144
20146,Zylog Systems Ltd.,275793.0,ZYLOG,2016-11-23,4.25,-0.022989,0.015807,1.118596,1.134402,0.640072,1.506896,-0.030786,0.004640,0.014079,0.008541,1.126326e-01,0.000961,0.047206,1.977908,-0.072004,0.110227,1.658702,-0.103254,0.172820,2.689463,0.087481,0.236579,3.498012,0.036935,-0.097569,-0.196195,-0.072971


In [50]:
output180CAR_2.to_pickle(rf"{output_folder_path}\output180CAR_2.pkl")

In [51]:
del ols180
del ols180CAR

del result180CAR2
del result180CAR

del output180CAR
del output180CAR_2

## 210 CAR Data Reading

In [52]:
ols210 = pd.read_pickle(rf"{import_folder_path}\ols210_2.pkl")

## 210 CAR Calc

In [53]:
ols210CAR = ols210.sort_values(by = ["CompanyName", "AsOnDate"])
ols210CAR

,CompanyName,ProwessCode,Symbol,AsOnDate,ACP,pct,RF,RMRF,MF,SMB,HML,OLS210_intercept,OLS210_RMRF,OLS210_r_squared,OLS210_adjusted_r_squared,OLS210_f_p_value
0,20 Microns Ltd.,11.0,20MICRONS,2008-10-06,16.82,NaN,0.069713,-6.381449,-6.311735,-0.373052,-0.566450,NaN,NaN,NaN,NaN,NaN
1,20 Microns Ltd.,11.0,20MICRONS,2008-10-07,15.05,-0.105232,0.023232,-0.669144,-0.645911,-1.502487,0.184699,NaN,NaN,NaN,NaN,NaN
2,20 Microns Ltd.,11.0,20MICRONS,2008-10-08,13.25,-0.119601,0.023232,-3.533362,-3.510130,-1.780674,0.072932,NaN,NaN,NaN,NaN,NaN
3,20 Microns Ltd.,11.0,20MICRONS,2008-10-10,11.60,-0.124528,0.045520,-7.052324,-7.006804,0.217126,0.354629,NaN,NaN,NaN,NaN,NaN
4,20 Microns Ltd.,11.0,20MICRONS,2008-10-13,12.32,0.062069,0.066863,5.042738,5.109602,-2.437097,0.145853,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13465088,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-21,0.35,0.000000,0.018215,1.441807,1.460021,0.572866,1.270303,NaN,NaN,NaN,NaN,NaN
13465089,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-22,0.35,0.000000,0.018215,0.593324,0.611538,0.752002,0.124527,NaN,NaN,NaN,NaN,NaN
13465090,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-26,0.35,0.000000,0.072878,-0.044514,0.028364,-1.091597,0.440605,NaN,NaN,NaN,NaN,NaN
13465091,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-27,0.35,0.000000,0.018215,0.326702,0.344916,-0.165072,-0.188505,NaN,NaN,NaN,NaN,NaN


In [54]:
# FUNCTION

# def CAR210(frame):
#     if ((~frame.OLS210_intercept.isnull().any()) == True):
#         ER = frame.RF + (frame.RMRF * frame.OLS210_RMRF) + (frame.SMB * frame.OLS210_SMB) + (frame.HML * frame.OLS210_HML) + frame.OLS210_intercept
#         AR = frame.pct - ER
#         return AR.sum()
#     else: 
#         return np.NaN

    # for i in range(len(frame)):
    #     if (i>0) & ( i< len(frame)-1) :
    #         coeff210CAR = np.array([frame.iloc[i]["OLS210_RMRF"], frame.iloc[i]["OLS210_SMB"],
    #                                 frame.iloc[i]["OLS210_HML"], 1, frame.iloc[i]["OLS210_intercept"]])

    #         event210CAR3 = np.array(frame.loc[ [i-1, i, i+1], ["RMRF", "SMB", "HML", "RF"]],)
    #         event210CAR3 = [np.append(item, 1) for item in event210CAR3]

    #         ER = np.matmul(event210CAR3, coeff210CAR)

    #         actualReturns = np.array([frame.iloc[i-1]["pct"], frame.iloc[i]["pct"], frame.iloc[i+1]["pct"]])

    #         AR = np.subtract( actualReturns, ER )

    #         output210CAR3.loc[i, ["210CAR3"]] = AR.sum()


    
        # if i>=2:
        #     frame["210CAR5"].iloc[i] = CAR210(frame.iloc[ i-2 : i+2 ])
        # if i>=3:
        #     frame["210CAR7"].iloc[i] = CAR210(frame.iloc[ i-3 : i+3 ])
        # if i>=5:
        #     frame["210CAR11"].iloc[i] = CAR210(frame.iloc[ i-5 : i+5 ])

        

def CAR(frame):

    outputFrame = pd.DataFrame( index = frame.index)

        
    # 3 Days
    
    outputFrame["pct_cen3sum"] = frame["pct"].rolling(window = 3, min_periods = 3, center = True).sum()
    
    outputFrame["RF_cen3sum"] = frame["RF"].rolling(window = 3, min_periods = 3, center = True).sum()
    
    outputFrame["RMRF_cen3sum"] = frame["RMRF"].rolling(window = 3, min_periods = 3, center = True).sum()
    
    
    # 5 Days
    
    outputFrame["pct_cen5sum"] = frame["pct"].rolling(window = 5, min_periods = 5, center = True).sum()
    
    outputFrame["RF_cen5sum"] = frame["RF"].rolling(window = 5, min_periods = 5, center = True).sum()
    
    outputFrame["RMRF_cen5sum"] = frame["RMRF"].rolling(window = 5, min_periods = 5, center = True).sum()
    
    
    # 7 Days
    
    outputFrame["pct_cen7sum"] = frame["pct"].rolling(window = 7, min_periods = 7, center = True).sum()
    
    outputFrame["RF_cen7sum"] = frame["RF"].rolling(window = 7, min_periods = 7, center = True).sum()
    
    outputFrame["RMRF_cen7sum"] = frame["RMRF"].rolling(window = 7, min_periods = 7, center = True).sum()
    
    
    # 11 Days
    
    outputFrame["pct_cen11sum"] = frame["pct"].rolling(window = 11, min_periods = 11, center = True).sum()
    
    outputFrame["RF_cen11sum"] = frame["RF"].rolling(window = 11, min_periods = 11, center = True).sum()
    
    outputFrame["RMRF_cen11sum"] = frame["RMRF"].rolling(window = 11, min_periods = 11, center = True).sum()
    
    
    return outputFrame


In [55]:
result210CAR = ols210CAR.sort_values(by = ["CompanyName", "AsOnDate"]).groupby(by = "CompanyName").progress_apply(CAR)

  0%|          | 0/3721 [00:00<?, ?it/s]

In [56]:
result210CAR

pct_cen3sum  RF_cen3sum  RMRF_cen3sum  \
CompanyName                                                          
20 Microns Ltd.    0                 NaN         NaN           NaN   
                   1                 NaN    0.116178    -10.583954   
                   2           -0.349361    0.091985    -11.254830   
                   3           -0.182061    0.135616     -5.542948   
                   4           -0.163920    0.134667     -0.834193   
...                                  ...         ...           ...   
Zylog Systems Ltd. 13465088     0.000000    0.054643      2.222872   
                   13465089     0.000000    0.109307      1.990617   
                   13465090     0.000000    0.109307      0.875511   
                   13465091     0.000000    0.109307      0.985242   
                   13465092          NaN         NaN           NaN   

                             pct_cen5sum  RF_cen5sum  RMRF_cen5sum  \
CompanyName                                                          
20 Microns Ltd.    0                 NaN         NaN           NaN   
                   1                 NaN         NaN           NaN   
                   2                 NaN    0.228562    -12.593540   
                   3           -0.388754    0.181132     -5.036699   
                   4           -0.183251    0.180182     -9.543586   
...                                  ...         ...           ...   
Zylog Systems Ltd. 13465088     0.000000    0.145734      0.669402   
                   13465089     0.000000    0.145735      2.505059   
                   13465090     0.000000    0.145736      3.020372   
                   13465091          NaN         NaN           NaN   
                   13465092          NaN         NaN           NaN   

                             pct_cen7sum  RF_cen7sum  RMRF_cen7sum  \
CompanyName                                                          
20 Microns Ltd.    0                 NaN         NaN           NaN   
                   1                 NaN         NaN           NaN   
                   2                 NaN         NaN           NaN   
                   3                 NaN    0.273128    -16.594178   
                   4           -0.163688    0.225697    -12.320480   
...                                  ...         ...           ...   
Zylog Systems Ltd. 13465088     0.000000    0.218599      1.086124   
                   13465089     0.000000    0.182163      1.699158   
                   13465090          NaN         NaN           NaN   
                   13465091          NaN         NaN           NaN   
                   13465092          NaN         NaN           NaN   

                             pct_cen11sum  RF_cen11sum  RMRF_cen11sum  
CompanyName                                                            
20 Microns Ltd.    0                  NaN          NaN            NaN  
                   1                  NaN          NaN            NaN  
                   2                  NaN          NaN            NaN  
                   3                  NaN          NaN            NaN  
                   4                  NaN          NaN            NaN  
...                                   ...          ...            ...  
Zylog Systems Ltd. 13465088           NaN          NaN            NaN  
                   13465089           NaN          NaN            NaN  
                   13465090           NaN          NaN            NaN  
                   13465091           NaN          NaN            NaN  
                   13465092           NaN          NaN            NaN  

[13465093 rows x 12 columns]

In [57]:
result210CAR2 = result210CAR.reset_index(level = 0).drop("CompanyName", axis = 1)
result210CAR2

,pct_cen3sum,RF_cen3sum,RMRF_cen3sum,pct_cen5sum,RF_cen5sum,RMRF_cen5sum,pct_cen7sum,RF_cen7sum,RMRF_cen7sum,pct_cen11sum,RF_cen11sum,RMRF_cen11sum
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,0.116178,-10.583954,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.349361,0.091985,-11.254830,NaN,0.228562,-12.593540,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.182061,0.135616,-5.542948,-0.388754,0.181132,-5.036699,NaN,0.273128,-16.594178,NaN,NaN,NaN
4,-0.163920,0.134667,-0.834193,-0.183251,0.180182,-9.543586,-0.163688,0.225697,-12.320480,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
13465088,0.000000,0.054643,2.222872,0.000000,0.145734,0.669402,0.000000,0.218599,1.086124,NaN,NaN,NaN
13465089,0.000000,0.109307,1.990617,0.000000,0.145735,2.505059,0.000000,0.182163,1.699158,NaN,NaN,NaN
13465090,0.000000,0.109307,0.875511,0.000000,0.145736,3.020372,NaN,NaN,NaN,NaN,NaN,NaN
13465091,0.000000,0.109307,0.985242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
output210CAR = pd.concat([ols210CAR, result210CAR2], axis = 1)
output210CAR

,CompanyName,ProwessCode,Symbol,AsOnDate,ACP,pct,RF,RMRF,MF,SMB,HML,OLS210_intercept,OLS210_RMRF,OLS210_r_squared,OLS210_adjusted_r_squared,OLS210_f_p_value,pct_cen3sum,RF_cen3sum,RMRF_cen3sum,pct_cen5sum,RF_cen5sum,RMRF_cen5sum,pct_cen7sum,RF_cen7sum,RMRF_cen7sum,pct_cen11sum,RF_cen11sum,RMRF_cen11sum
0,20 Microns Ltd.,11.0,20MICRONS,2008-10-06,16.82,NaN,0.069713,-6.381449,-6.311735,-0.373052,-0.566450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20 Microns Ltd.,11.0,20MICRONS,2008-10-07,15.05,-0.105232,0.023232,-0.669144,-0.645911,-1.502487,0.184699,NaN,NaN,NaN,NaN,NaN,NaN,0.116178,-10.583954,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20 Microns Ltd.,11.0,20MICRONS,2008-10-08,13.25,-0.119601,0.023232,-3.533362,-3.510130,-1.780674,0.072932,NaN,NaN,NaN,NaN,NaN,-0.349361,0.091985,-11.254830,NaN,0.228562,-12.593540,NaN,NaN,NaN,NaN,NaN,NaN
3,20 Microns Ltd.,11.0,20MICRONS,2008-10-10,11.60,-0.124528,0.045520,-7.052324,-7.006804,0.217126,0.354629,NaN,NaN,NaN,NaN,NaN,-0.182061,0.135616,-5.542948,-0.388754,0.181132,-5.036699,NaN,0.273128,-16.594178,NaN,NaN,NaN
4,20 Microns Ltd.,11.0,20MICRONS,2008-10-13,12.32,0.062069,0.066863,5.042738,5.109602,-2.437097,0.145853,NaN,NaN,NaN,NaN,NaN,-0.163920,0.134667,-0.834193,-0.183251,0.180182,-9.543586,-0.163688,0.225697,-12.320480,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13465088,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-21,0.35,0.000000,0.018215,1.441807,1.460021,0.572866,1.270303,NaN,NaN,NaN,NaN,NaN,0.000000,0.054643,2.222872,0.000000,0.145734,0.669402,0.000000,0.218599,1.086124,NaN,NaN,NaN
13465089,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-22,0.35,0.000000,0.018215,0.593324,0.611538,0.752002,0.124527,NaN,NaN,NaN,NaN,NaN,0.000000,0.109307,1.990617,0.000000,0.145735,2.505059,0.000000,0.182163,1.699158,NaN,NaN,NaN
13465090,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-26,0.35,0.000000,0.072878,-0.044514,0.028364,-1.091597,0.440605,NaN,NaN,NaN,NaN,NaN,0.000000,0.109307,0.875511,0.000000,0.145736,3.020372,NaN,NaN,NaN,NaN,NaN,NaN
13465091,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-27,0.35,0.000000,0.018215,0.326702,0.344916,-0.165072,-0.188505,NaN,NaN,NaN,NaN,NaN,0.000000,0.109307,0.985242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
output210CAR.columns

Index(['CompanyName', 'ProwessCode', 'Symbol', 'AsOnDate', 'ACP', 'pct', 'RF',
       'RMRF', 'MF', 'SMB', 'HML', 'OLS210_intercept', 'OLS210_RMRF',
       'OLS210_r_squared', 'OLS210_adjusted_r_squared', 'OLS210_f_p_value',
       'pct_cen3sum', 'RF_cen3sum', 'RMRF_cen3sum', 'pct_cen5sum',
       'RF_cen5sum', 'RMRF_cen5sum', 'pct_cen7sum', 'RF_cen7sum',
       'RMRF_cen7sum', 'pct_cen11sum', 'RF_cen11sum', 'RMRF_cen11sum'],
      dtype='object')

In [60]:
output210CAR["210CAR3"] = output210CAR["pct_cen3sum"] - (output210CAR["RF_cen3sum"] + 
                          
                          output210CAR["OLS210_RMRF"]*output210CAR["RMRF_cen3sum"] + 
                          output210CAR["OLS210_intercept"]*3 )


In [61]:
output210CAR["210CAR5"] = output210CAR["pct_cen5sum"] - (output210CAR["RF_cen5sum"] + 
                          
                          output210CAR["OLS210_RMRF"]*output210CAR["RMRF_cen5sum"] + 
                          output210CAR["OLS210_intercept"]*3 )

In [62]:
output210CAR["210CAR7"] = output210CAR["pct_cen7sum"] - (output210CAR["RF_cen7sum"] + 
                          
                          output210CAR["OLS210_RMRF"]*output210CAR["RMRF_cen7sum"] + 
                          output210CAR["OLS210_intercept"]*3 )

In [63]:
output210CAR["210CAR11"] = output210CAR["pct_cen11sum"] - (output210CAR["RF_cen11sum"] + 
                          
                          output210CAR["OLS210_RMRF"]*output210CAR["RMRF_cen11sum"] + 
                          output210CAR["OLS210_intercept"]*3 )

In [64]:
output210CAR

,CompanyName,ProwessCode,Symbol,AsOnDate,ACP,pct,RF,RMRF,MF,SMB,HML,OLS210_intercept,OLS210_RMRF,OLS210_r_squared,OLS210_adjusted_r_squared,OLS210_f_p_value,pct_cen3sum,RF_cen3sum,RMRF_cen3sum,pct_cen5sum,RF_cen5sum,RMRF_cen5sum,pct_cen7sum,RF_cen7sum,RMRF_cen7sum,pct_cen11sum,RF_cen11sum,RMRF_cen11sum,210CAR3,210CAR5,210CAR7,210CAR11
0,20 Microns Ltd.,11.0,20MICRONS,2008-10-06,16.82,NaN,0.069713,-6.381449,-6.311735,-0.373052,-0.566450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20 Microns Ltd.,11.0,20MICRONS,2008-10-07,15.05,-0.105232,0.023232,-0.669144,-0.645911,-1.502487,0.184699,NaN,NaN,NaN,NaN,NaN,NaN,0.116178,-10.583954,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20 Microns Ltd.,11.0,20MICRONS,2008-10-08,13.25,-0.119601,0.023232,-3.533362,-3.510130,-1.780674,0.072932,NaN,NaN,NaN,NaN,NaN,-0.349361,0.091985,-11.254830,NaN,0.228562,-12.593540,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20 Microns Ltd.,11.0,20MICRONS,2008-10-10,11.60,-0.124528,0.045520,-7.052324,-7.006804,0.217126,0.354629,NaN,NaN,NaN,NaN,NaN,-0.182061,0.135616,-5.542948,-0.388754,0.181132,-5.036699,NaN,0.273128,-16.594178,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20 Microns Ltd.,11.0,20MICRONS,2008-10-13,12.32,0.062069,0.066863,5.042738,5.109602,-2.437097,0.145853,NaN,NaN,NaN,NaN,NaN,-0.163920,0.134667,-0.834193,-0.183251,0.180182,-9.543586,-0.163688,0.225697,-12.320480,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13465088,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-21,0.35,0.000000,0.018215,1.441807,1.460021,0.572866,1.270303,NaN,NaN,NaN,NaN,NaN,0.000000,0.054643,2.222872,0.000000,0.145734,0.669402,0.000000,0.218599,1.086124,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13465089,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-22,0.35,0.000000,0.018215,0.593324,0.611538,0.752002,0.124527,NaN,NaN,NaN,NaN,NaN,0.000000,0.109307,1.990617,0.000000,0.145735,2.505059,0.000000,0.182163,1.699158,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13465090,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-26,0.35,0.000000,0.072878,-0.044514,0.028364,-1.091597,0.440605,NaN,NaN,NaN,NaN,NaN,0.000000,0.109307,0.875511,0.000000,0.145736,3.020372,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13465091,Zylog Systems Ltd.,275793.0,ZYLOG,2024-03-27,0.35,0.000000,0.018215,0.326702,0.344916,-0.165072,-0.188505,NaN,NaN,NaN,NaN,NaN,0.000000,0.109307,0.985242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
output210CAR_2 = output210CAR.dropna(subset = "210CAR3").reset_index(drop = True)
output210CAR_2

,CompanyName,ProwessCode,Symbol,AsOnDate,ACP,pct,RF,RMRF,MF,SMB,HML,OLS210_intercept,OLS210_RMRF,OLS210_r_squared,OLS210_adjusted_r_squared,OLS210_f_p_value,pct_cen3sum,RF_cen3sum,RMRF_cen3sum,pct_cen5sum,RF_cen5sum,RMRF_cen5sum,pct_cen7sum,RF_cen7sum,RMRF_cen7sum,pct_cen11sum,RF_cen11sum,RMRF_cen11sum,210CAR3,210CAR5,210CAR7,210CAR11
0,20 Microns Ltd.,11.0,20MICRONS,2011-04-29,20.48,-0.016330,0.019851,-0.595940,-0.576089,-0.572908,-0.607305,-0.023121,0.009385,0.073740,0.069287,6.696283e-05,0.135960,0.099051,-2.055273,0.068979,0.138538,-4.268887,0.048651,0.178026,-4.607642,0.066696,0.316887,-5.076389,0.125561,0.039867,-0.016769,-0.133185
1,20 Microns Ltd.,11.0,20MICRONS,2014-08-06,31.15,-0.011111,0.022705,-0.690542,-0.667837,1.023324,-0.182707,-0.035117,0.005551,0.026592,0.021912,1.803598e-02,-0.012716,0.068115,-0.250139,-0.004716,0.158951,-0.558310,-0.002971,0.249787,-1.450041,-0.040029,0.363107,-1.042710,0.025908,-0.055217,-0.139358,-0.291996
2,20 Microns Ltd.,11.0,20MICRONS,2017-05-04,43.70,0.016279,0.016554,0.046274,0.062828,0.831535,-0.581499,-0.024740,0.017188,0.166374,0.162366,7.998804e-10,-0.023826,0.049554,-0.994942,-0.031524,0.165027,-0.704433,-0.043378,0.198027,-0.312208,-0.078551,0.263922,0.266193,0.017940,-0.110223,-0.161819,-0.272828
3,20 Microns Ltd.,11.0,20MICRONS,2019-05-28,41.00,-0.016787,0.016873,0.367905,0.384778,-0.177790,0.292518,-0.025401,0.017863,0.233449,0.229763,1.106707e-13,0.060421,0.084375,0.593600,0.127139,0.117695,2.441371,0.116773,0.151016,2.141790,0.099791,0.250771,1.525870,0.041646,0.042037,0.003702,-0.102033
4,20 Microns Ltd.,11.0,20MICRONS,2023-05-16,82.20,-0.000608,0.018393,-0.219663,-0.201270,0.165950,0.576497,-0.023010,0.017708,0.228941,0.225234,2.055635e-13,-0.019717,0.091975,-0.425848,-0.001636,0.128482,-1.382064,0.011033,0.164989,-0.852851,0.042175,0.273984,0.631503,-0.035121,-0.036613,-0.069823,-0.173961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20031,Zylog Systems Ltd.,275793.0,ZYLOG,2014-02-14,11.40,0.004405,0.023865,0.579200,0.603064,-0.772576,0.455377,-0.039630,0.012446,0.069711,0.065238,1.078451e-04,-0.021532,0.119340,-0.605400,-0.056325,0.166753,-0.114553,-0.073061,0.214166,0.387515,-0.100032,0.356106,0.623586,-0.014447,-0.102762,-0.173160,-0.345009
20032,Zylog Systems Ltd.,275793.0,ZYLOG,2014-11-19,7.70,-0.025316,0.021860,-0.741569,-0.719709,0.280029,-0.546563,-0.037284,0.014662,0.059576,0.055055,3.567988e-04,-0.166621,0.065476,-0.584795,-0.128630,0.152826,0.041803,-0.054409,0.239965,0.676165,-0.144981,0.327195,-0.398142,-0.111669,-0.170216,-0.192435,-0.354486
20033,Zylog Systems Ltd.,275793.0,ZYLOG,2015-08-14,5.40,0.018868,0.019213,1.546309,1.565522,-0.064032,1.416502,-0.035294,0.013010,0.077200,0.072764,4.444277e-05,-0.113635,0.096076,0.535057,-0.246220,0.134926,-0.534668,-0.291712,0.173777,-0.844522,-0.274151,0.291613,-3.755096,-0.110791,-0.268309,-0.348619,-0.411028
20034,Zylog Systems Ltd.,275793.0,ZYLOG,2016-11-23,4.25,-0.022989,0.015807,1.118596,1.134402,0.640072,1.506896,-0.027389,0.004182,0.010090,0.005331,1.468822e-01,0.000961,0.047206,1.977908,-0.072004,0.110227,1.658702,-0.103254,0.172820,2.689463,0.087481,0.236579,3.498012,0.027651,-0.106999,-0.205153,-0.081559


In [66]:
output210CAR_2.to_pickle(rf"{output_folder_path}\output210CAR_2.pkl")